In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
# -----------------------------
# SET YOUR FOLDER PATHS HERE
bare_pcb_folder = "/kaggle/input/pcb-defect-dataset/pcb-defect-dataset/train/images"        # folder with bare PCBs
solder_pcb_folder = "/kaggle/input/soldef-ai-pcb-dataset-for-defect-detection/SolDef_AI/Labeled"     # folder with solder PCBs

# -----------------------------
# Load images from folder (ignore .txt files)
def load_images_from_folder(folder_path, label, image_size=(224, 224)):
    images = []
    labels = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Only image files
            file_path = os.path.join(folder_path, filename)
            try:
                img = load_img(file_path, target_size=image_size)
                img_array = img_to_array(img) / 255.0  # normalize
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Could not load {file_path}: {e}")
    return images, labels

# -----------------------------
# Load bare PCB and solder PCB images
bare_images, bare_labels = load_images_from_folder(bare_pcb_folder, label=0)
solder_images, solder_labels = load_images_from_folder(solder_pcb_folder, label=1)

# Combine the two datasets
X = np.array(bare_images + solder_images)
y = np.array(bare_labels + solder_labels)

print(f"Total images loaded: {len(X)}")
print(f"Labels: 0 = bare, 1 = solder")

# -----------------------------
# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# -----------------------------
# Build CNN Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # Binary classification output
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# -----------------------------
# Train the Model
history = model.fit(X_train, y_train, epochs=1, validation_data=(X_test, y_test), batch_size=32)

# -----------------------------
# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"\n Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# -----------------------------
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
# plt.legend()
# plt.grid(True)
plt.show()